In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from textwrap import indent
from IPython.display import Javascript
from liblet import ANTLR, AnnotatedTreeWalker, Tree, pyast2tree, side_by_side
from IPython.display import Latex
from fractions import Fraction
import importlib
from core.arithmetic_manager import ArithManager
from core.util import *
from core.shunting_yard_parser import ShuntingYardParser
from core.arithmetic_manager import ArithManager
from core.util import *

In [3]:
def ast_eval(ast, domain, show_blocks, show_latex):
    AM = ArithManager()
    
    blocks = AM.blocks(ast)
    
    if show_blocks: print(blocks)
        
    MEMORY = dict(blocks)
    main_block = blocks[-1][1]
    child_to_calc = None
    current_block = block_id = None

    while blocks and is_calculable(blocks[0][1]): blocks = blocks[1:]

    while blocks:
        block_id, current_block = blocks[0]
        parent_to_calc = AM.prior(current_block)
        tex = AM.latex(main_block, MEMORY)
        if show_latex: display_latex(tex)
        
        parent_to_calc.children =  [Tree({'type': 'atomExpr', 'value': AM.eval(child, domain, MEMORY), 'priority': 0, '_calc': 'last'}, []) 
                                    if is_next_to_calc(child) else child
                                    for child in parent_to_calc.children]

        if is_calculable(current_block):
            current_block = current_block.children[0]
            blocks = blocks[1:]

        MEMORY[block_id] = current_block

    tex = AM.latex(main_block, {})
    if show_latex: display_latex(tex)
    
    return main_block.children[0].root['value']



def shunting_eval(expr, domain, show_ast = False, show_blocks = False, show_latex = False):
    AM = ArithManager()

    ast = AM.shuntingYardExpr2ast(expr)
    if show_ast: display(ast)

    return ast_eval(ast, domain, show_blocks, show_latex)

In [7]:
from core.expression_generator import generate_expression

wrong_expr = []
for _ in range(30):
    exception = False
    expr = generate_expression(3)

    try:
        shunting_res = shunting_eval(expr, 'R')
        if exception: 
            wrong_expr.append(expr)
            continue
    except Exception as e:
        print('** EXCEPTION SHUNTING ** ', e)
        if not exception: 
            wrong_expr.append(expr)
        continue

    formatted_expr = expr.replace('x', '*').replace(':', '/').replace('^', '**')
    res =  round(eval(formatted_expr), 3)

    if abs(shunting_res - res) > 1:
        wrong_expr.append(expr)
        
    
    #print(antlr_res, shunting_res, res)
        
if wrong_expr != []:
    print(f'*** WRONG {len(wrong_expr)} ***\n')
    for i, it in enumerate(wrong_expr):
        print(f' $$$ {i} $$$  ', it)
else:
    print("ALL DONE")

** EXCEPTION SHUNTING **  DOMINIO ERRATO!!!!!
** EXCEPTION SHUNTING **  DOMINIO ERRATO!!!!!
** EXCEPTION SHUNTING **  DOMINIO ERRATO!!!!!
** EXCEPTION SHUNTING **  DOMINIO ERRATO!!!!!
*** WRONG 4 ***

 $$$ 0 $$$   - 96 - 16 ^ 2 ^ -2 / - 48 - 41 ^ 2 x - 51 - 37 + 78 / 15 / 88 - 68 - 93 x 82 ^ -2 ^ -2 / - - 62 ^ 2 ^ 2 - 85 / 19 - 22 / 9 ^ -2 - - 79 x 79 - 82 / - 24 + 63 x 97 - - 74 + 93 / - 63 - - 3 ^ -2 x 83 / 6 - 11 ^ -2 + - 16 - 24 / 71 - 91 ^ 2 / - - - 69 / 44 - - 11 ^ -2 x - 40 / 23 - - 96 x - - 49 x - 66 ^ 2 x - 94 x 77 + 98 / 32 ^ -2 x 58 / 98 x 3 - - 1 x 35 ^ 2 + 56 x - 18 / 11 - 48 - 40 / - - 97 - 57 + - - - 24 / 9 + - 69 - 31 + - - 89 + - - - - 5 - - - - - 59 / - 16 x 23 / - - - 61 x 38 x 62 - 45 / 34 + 29 + 13 ^ -2 / 61 - - - 11 - - - 8 / 39 + 80 x 85 ^ 2 / 58 + 1 + - 29 - 54 ^ 2 - - - 3 ^ 2 - 49 x 13 x 43 / 63 ^ -2 ^ -2
 $$$ 1 $$$   - - - 30 / 2 x - 4 ^ -2 + - - 96 x 3 - - 97 ^ -2 ^ 2 / - 69 ^ 2 ^ 2 ^ -2 + 80 + 43 / 69 + 27 + 50 - - 32 ^ -2 ^ 2 x 77 + 83 ^ -2 ^ 2 + - - 77 + 3

In [37]:
expr = ' - - - 10 / - 27 x - - 8 x 67 + 84 x - 49 / - 88 + 60 ^ 2 + 72 x 81 ^ 2 / 82 + 51 + 40 / 69 ^ 2 / - - - 41 - - 93 ^ -2 / - - 91 ^ 2 ^ -2 x - - 64 + - 27 - - 18 + - 2 x 32 x 69 + 23 x 25 x 12 - - 97 x - 70 x - - 22 + 95 x - 31 / 39 + 35 + 82 / 74 + 45 ^ 2'
formatted_expr = expr.replace('x', '*').replace(':', '/').replace('^', '**')
real_expr = expr.replace('x', '*').replace(':', '/')
print(real_expr)
res =  round(eval(formatted_expr), 7)
shunt = round(float(shunting_eval(expr, 'R', show_latex=False)), 7)
print(shunt, '\n', res)

 - - - 10 / - 27 * - - 8 * 67 + 84 * - 49 / - 88 + 60 ^ 2 + 72 * 81 ^ 2 / 82 + 51 + 40 / 69 ^ 2 / - - - 41 - - 93 ^ -2 / - - 91 ^ 2 ^ -2 * - - 64 + - 27 - - 18 + - 2 * 32 * 69 + 23 * 25 * 12 - - 97 * - 70 * - - 22 + 95 * - 31 / 39 + 35 + 82 / 74 + 45 ^ 2
-135262.2332269 
 -135262.2332269


In [34]:
token = '-3.743464808816791e-05'
if re.match(r'-?[0-9]+\.[0-9]+$', token):
    print("YES")
    

    
10.4**40

4.801020627936657e+40